In [1]:
import os
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

模型的保存与恢复。这里同样先构建好计算图，模型的保存代码写在训练过程中。

In [2]:
import tensorflow as tf
from dataset.dataset import load_cifar10


train_data, test_data = load_cifar10(batch_size=64)

unit_I = train_data.n_features

filters = 32
conv_size = (3, 3)

pool_size = (2, 2)
strides = (2, 2)

unit_O = 10

X = tf.placeholder(tf.float32, [None, unit_I])
Y = tf.placeholder(tf.int64, [None])
X_img = tf.transpose(tf.reshape(
    X, [-1, 3, 32, 32]), perm=[0, 2, 3, 1])

with tf.name_scope('CNN'):
    conv1 = tf.layers.conv2d(X_img, filters=filters,
                             kernel_size=conv_size, padding='same',
                             activation=tf.nn.relu, name='conv1')
    pooling1 = tf.layers.max_pooling2d(conv1, pool_size=pool_size,
                                       strides=strides, name='pooling1')
    conv2 = tf.layers.conv2d(pooling1, filters=filters,
                             kernel_size=conv_size, padding='same',
                             activation=tf.nn.relu, name='conv2')
    pooling2 = tf.layers.max_pooling2d(conv2, pool_size=pool_size,
                                       strides=strides, name='pooling2')
    conv3 = tf.layers.conv2d(pooling2, filters=filters,
                             kernel_size=conv_size, padding='same',
                             activation=tf.nn.relu, name='conv3')
    pooling3 = tf.layers.max_pooling2d(conv3, pool_size=pool_size,
                                       strides=strides, name='pooling3')
    logits = tf.layers.dense(tf.layers.flatten(
        pooling3), unit_O, activation=None)

with tf.name_scope('Eval'):
    loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
    predict = tf.argmax(logits, 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), tf.float32))

with tf.name_scope('train_op'):
    lr = 1e-3
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)

init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(50000, 3072) (50000,)


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(10000, 3072) (10000,)
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Instructions for updating:
Use tf.cast instead.


指定模型保存的目录：：

In [3]:
CUR_DIR = '.'
save_dir = os.path.join(CUR_DIR, 'model')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

saved_model = 'ckp-16'    # 当存在第16个epoch检查点时

在训练代码中加入saver，为了模拟训练过程的断点与恢复，当训练进行到第16个epoch之后时手动终止程序，然后再运行一边程序查看输出。

In [4]:
with tf.Session(config=config) as sess:
    sess.run(init)
    epochs = 20

    # 创建saver
    saver = tf.train.Saver()

    model_path = os.path.join(save_dir, saved_model)
    if os.path.exists(model_path+'.index'):    # 如果指定的检查点存在
        saver.restore(sess, model_path)
        print('restore from {}'.format(saved_model))
        trained_epoch = 15    # 已完成的训练轮次
    else:
        trained_epoch = 0

    batch_cnt = 0
    for epoch in range(epochs-trained_epoch):
        for batch_data, batch_labels in train_data.next_batch():
            loss_val, acc_val, _ = sess.run(
                [loss, accuracy, train_op],
                feed_dict={
                    X: batch_data,
                    Y: batch_labels})
            batch_cnt += 1

            # 每1000个batch保存一次模型
            if (batch_cnt+1) % 500 == 0:
                saver.save(sess,
                           os.path.join(save_dir, 'ckp-{}'.format(epoch+trained_epoch+1)))    # 以epoch为单位生成文件
                print('model saved to ckp-{}'.format(epoch+trained_epoch+1))

    # 训练完毕后还需要保存一次模型
    saver.save(sess, os.path.join(
        save_dir, 'ckp-{}'.format(epoch+trained_epoch+1)))
    print('model saved to ckp-{}'.format(epoch+trained_epoch+1))
    print('training done.')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/ckp-16
restore from ckp-16
model saved to ckp-16
model saved to ckp-17
model saved to ckp-17
model saved to ckp-18
model saved to ckp-19
model saved to ckp-19
model saved to ckp-20
model saved to ckp-20
training done.
